In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

Catboost is a boosting algorithm on decision trees.
Advantages:
No need to spend time on transformation of data(Feature Enginnering)
It can be trained on the GPU, so it is faster

In [3]:
def get_summary (df):
  print (f"Dataset Shape: {df. shape}")
  summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
  summary= summary.reset_index()
  summary['Name'] = summary['index']
  summary= summary [['Name', 'dtypes']] 
  summary['Missing'] = df.isnull().sum().values
  summary['Uniques '] = df.nunique().values
  summary['First Value'] = df.loc[0].values 
  summary['Second Value'] = df.loc[1].values
  return summary

In [4]:
df= pd.read_csv('loan_data_set.csv')

In [5]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,NaN,360.0,1.0,Y
1,4583,1508.0,128.0,360.0,1.0,N
2,3000,0.0,66.0,360.0,1.0,Y
3,2583,2358.0,120.0,360.0,1.0,Y
4,6000,0.0,141.0,360.0,1.0,Y


In [6]:
df.shape

(614, 6)

In [7]:
result=get_summary(df)
result

Dataset Shape: (614, 6)


,Name,dtypes,Missing,Uniques,First Value,Second Value
0,ApplicantIncome,int64,0,505,5849,4583
1,CoapplicantIncome,float64,0,287,0.0,1508.0
2,LoanAmount,float64,22,203,NaN,128.0
3,Loan_Amount_Term,float64,14,10,360.0,360.0
4,Credit_History,float64,50,2,1.0,1.0
5,Loan_Status,object,0,2,Y,N


In [9]:
df['Credit_History']=df['Credit_History'].astype(str)


In [10]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,NaN,360.0,1.0,Y
1,4583,1508.0,128.0,360.0,1.0,N
2,3000,0.0,66.0,360.0,1.0,Y
3,2583,2358.0,120.0,360.0,1.0,Y
4,6000,0.0,141.0,360.0,1.0,Y


In [11]:
target_col = "Loan_Status"
X=df.loc[:, df.columns !=target_col]
y=df.loc[:, target_col]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state= 42)
print(f"Train Data: {X_train.shape},{y_train.shape}")
print(f"Test Data: {X_test.shape},{y_test.shape}")

Train Data: (411, 5),(411,)
Test Data: (203, 5),(203,)


In [13]:
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
244,3406,4417.0,123.0,360.0,1.0
393,1993,1625.0,113.0,180.0,1.0
310,2917,0.0,84.0,360.0,1.0
408,8300,0.0,152.0,300.0,0.0
572,16666,0.0,275.0,360.0,1.0


In [14]:
features=list(X_train.columns)
features

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [15]:
cat_features=['Credit_History']

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ApplicantIncome    614 non-null    int64  
 1   CoapplicantIncome  614 non-null    float64
 2   LoanAmount         592 non-null    float64
 3   Loan_Amount_Term   600 non-null    float64
 4   Credit_History     614 non-null    object 
 5   Loan_Status        614 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 28.9+ KB


In [17]:
model_cb = CatBoostClassifier(task_type='GPU', iterations=100, random_state=2021, eval_metric="F1")

In [18]:
model_cb.fit(X_train, y_train, cat_features=cat_features, plot=True, eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.199227
0:	learn: 0.8401163	test: 0.7706422	best: 0.7706422 (0)	total: 6.94ms	remaining: 687ms
1:	learn: 0.8689024	test: 0.8376623	best: 0.8376623 (1)	total: 15.2ms	remaining: 745ms
2:	learn: 0.8788820	test: 0.8600000	best: 0.8600000 (2)	total: 21.2ms	remaining: 684ms
3:	learn: 0.8806202	test: 0.8628763	best: 0.8628763 (3)	total: 26.8ms	remaining: 644ms
4:	learn: 0.8765432	test: 0.8628763	best: 0.8628763 (3)	total: 32.6ms	remaining: 620ms
5:	learn: 0.8765432	test: 0.8628763	best: 0.8628763 (3)	total: 38.4ms	remaining: 602ms
6:	learn: 0.8765432	test: 0.8628763	best: 0.8628763 (3)	total: 44.4ms	remaining: 590ms
7:	learn: 0.8765432	test: 0.8628763	best: 0.8628763 (3)	total: 50.2ms	remaining: 577ms
8:	learn: 0.8778980	test: 0.8628763	best: 0.8628763 (3)	total: 57.1ms	remaining: 578ms
9:	learn: 0.8792570	test: 0.8628763	best: 0.8628763 (3)	total: 63.2ms	remaining: 569ms
10:	learn: 0.8778980	test: 0.8628763	best: 0.8628763 (3)	total: 68.8ms	remaining: 557ms
11:	learn: 0

In [19]:
y_pred=model_cb.predict(X_test)

In [25]:
df['Loan_Status'].isnull().sum()

0

In [28]:
accuracy_score(y_test, y_pred)

0.8029556650246306